In [1]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
# Load your features and labels datasets
features = pd.read_csv('data/features.csv')
labels = pd.read_csv('data/labels.csv')

# Check the structure of both dataframes to ensure they match
print(features.shape)  # Ensure the number of rows match in features and labels
print(labels.shape)

# Check for missing data or clean if necessary
print(features.isnull().sum())  # Check for missing values in features
print(labels.isnull().sum())    # Check for missing values in labels

(1521377, 8)
(1521377, 7)
latitude     0
longitude    0
cog          0
sog          0
rot          0
heading      0
anchored     0
time_diff    0
dtype: int64
latitude     0
longitude    0
cog          0
sog          0
rot          0
heading      0
anchored     0
dtype: int64


In [3]:
# Ensure the features and labels match in terms of rows
assert len(features) == len(labels), "Mismatch between features and labels row counts!"

# Split features and labels (assuming all columns in labels.csv are the target values)
X_train = features  # All columns in features are input features
y_train = labels  # Assuming all columns in labels.csv are the target (if not, select the correct one)

In [4]:
# Split the training data into training and validation sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create an XGBoost model instance
bst = XGBClassifier(n_estimators=100, max_depth=4, learning_rate=0.1, objective='binary:logistic')

# Fit the model on the training data
bst.fit(X_train, y_train)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [      0       1       2 ... 1474102 1474103 1474104], got [-167.54093 -167.48913 -167.46532 ...  178.751    178.76084  178.80538]

In [ ]:
# Otherwise, if evaluating on training data (not typical), you can use:
preds_train = bst.predict(X_train)
accuracy = accuracy_score(y_train, preds_train)
print(f"Training Accuracy: {accuracy}")